<a href="https://colab.research.google.com/github/ny-yo/kaggle-CommonLit-Readability-Prize/blob/main/03_commonlit_roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

pretrain済みモデルをロード

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import drive #インポート
drive.mount('/content/drive/') #GoogleDriveのマウント

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# download API key from google drive
## Original: https://colab.research.google.com/drive/1eufc8aNCdjHbrBhuy7M7X6BGyzAyRbrF#scrollTo=y5_288BYp6H1
## When you run for the first time, you will see a link to authenticate.

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/root/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [ ]:
!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes        200           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes        327           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        970           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       6020           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      50300            True  
house-prices-advanced-regression-techniques    2030-01-01 00:00:00  Getting Started  Knowledge      13153            True  
connectx

In [ ]:
!kaggle competitions download -c commonlitreadabilityprize

sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
test.csv: Skipping, found more recently modified local copy (use --force to force download)
train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

train = pd.read_csv("/content/train.csv.zip")
test = pd.read_csv("/content/test.csv")
sample= pd.read_csv("/content/sample_submission.csv")

print("train :", train.shape)
print("test  :", test.shape)
print("sample:", sample.shape)

train : (2834, 6)
test  : (7, 4)
sample: (7, 2)


In [ ]:
!pip install transformers

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
import torch
import torch.nn as nn
import transformers
import random
from torch.utils.data import DataLoader, Dataset
import warnings
warnings.simplefilter('ignore')

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import RobertaForSequenceClassification,RobertaTokenizer, AdamW, RobertaModel
from transformers.file_utils import ModelOutput
from transformers import get_linear_schedule_with_warmup

scaler = torch.cuda.amp.GradScaler() # GPUでの高速化。

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
#エポック数
epochs = 100
MODEL = "/content/drive/MyDrive/02-roberta_pretrain_base" #pretrain model path
train_batch = 16
valid_batch = 32
MAX_LEN = 314
SEED = 42

In [ ]:
train = train.sort_values("target").reset_index(drop=True)

In [ ]:
train["kfold"] = train.index % 5

In [ ]:
# Set seeds for reproducibility
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
tokenizer = RobertaTokenizer.from_pretrained(MODEL) 

In [ ]:
class RobertaDataset(Dataset):
    def __init__(self, sentences, targets):
        self.sentences = sentences
        self.targets = targets
    
    def __len__(self):
        return len(self.sentences)
    
    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        roberta_sentence = tokenizer.encode_plus(
                                    sentence,
                                    add_special_tokens = True,
                                    max_length = MAX_LEN,
                                    pad_to_max_length = True,
                                    return_attention_mask = True)
        
        ids = torch.tensor(roberta_sentence['input_ids'], dtype=torch.long)
        mask = torch.tensor(roberta_sentence['attention_mask'], dtype=torch.long)
        #token_type_ids = torch.tensor(roberta_sentence['token_type_ids'], dtype=torch.long)
        
        target = torch.tensor(self.targets[idx], dtype=torch.float)
        
        return{
            'ids':ids,
            'mask':mask,
            #'token_type_ids':token_type_ids,
            'target':target
        }

In [ ]:
class RegressorOutput(ModelOutput):
    loss = None
    logits = None
    hidden_states = None
    attentions = None

In [ ]:
#https://www.kaggle.com/vigneshbaskaran/commonlit-making-my-transformer-good-enough
class AttentionHead(nn.Module):
    def __init__(self, hidden_dim):
        super(AttentionHead, self).__init__()
        self.W = nn.Linear(hidden_dim, hidden_dim)
        self.V = nn.Linear(hidden_dim, 1)
    
    def forward(self, x):
        attention_scores = self.V(torch.tanh(self.W(x)))
        attention_scores = torch.softmax(attention_scores, dim=1)
        attentive_x = attention_scores * x
        attentive_x = attentive_x.sum(axis=1)
        return attentive_x

In [ ]:
#https://www.kaggle.com/vigneshbaskaran/commonlit-making-my-transformer-good-enough
class RobertaLastHiddenStateRegressor(nn.Module):
    def __init__(self, model_path):
        super(RobertaLastHiddenStateRegressor, self).__init__()
        self.roberta = RobertaModel.from_pretrained(model_path)
        self.head = AttentionHead(768)
        self.dropout = nn.Dropout(0.3)
        self.regressor = nn.Linear(768, 1)
        self.loss_fn = nn.MSELoss()
    
    def forward(self, input_ids, attention_mask, labels=None):
        roberta_outputs = self.roberta(input_ids=input_ids,
                                       attention_mask=attention_mask)
        last_hidden_state = roberta_outputs['last_hidden_state']
        attentive_vector = self.head(last_hidden_state)
        attentive_vector = self.dropout(attentive_vector)
        logits = self.regressor(attentive_vector)
        loss = torch.sqrt(self.loss_fn(labels, logits)) if labels is not None else None
        return RegressorOutput(loss=loss, logits=logits)

In [ ]:
def loss_fn(output,target):
    return torch.sqrt(nn.MSELoss()(output,target))

In [ ]:
def training(
    train_dataloader,
    model,
    optimizer,
    scheduler
):
    
    model.train()
    torch.backends.cudnn.benchmark = True

    allpreds = []
    alltargets = []

    for a in train_dataloader:

        losses = []

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():

            ids = a["ids"].to(device,non_blocking=True) # non_blocking=TrueでPinned MemoryからGPUに転送中もCPUが動作できるらしい。
            mask = a["mask"].to(device,non_blocking=True)
            #tokentype = a["token_type_ids"].to(device,non_blocking=True)

            output = model(ids,mask)
            output = output["logits"].squeeze(-1)

            target = a["target"].to(device,non_blocking=True)

            loss = loss_fn(output,target)


            # スコア習得用
            losses.append(loss.item())
            allpreds.append(output.detach().cpu().numpy())
            alltargets.append(target.detach().squeeze(-1).cpu().numpy())

        scaler.scale(loss).backward() # ロスのバックワード
        scaler.step(optimizer) # オプティマイザーの更新
        scaler.update() # スケーラーの更新
        
        del loss # ここでlossを消した方がGPUのメモリの無駄な部分を消せるらしい。https://tma15.github.io/blog/2020/08/22/pytorch%E4%B8%8D%E8%A6%81%E3%81%AB%E3%81%AA%E3%81%A3%E3%81%9F%E8%A8%88%E7%AE%97%E3%82%B0%E3%83%A9%E3%83%95%E3%82%92%E5%89%8A%E9%99%A4%E3%81%97%E3%81%A6%E3%83%A1%E3%83%A2%E3%83%AA%E3%82%92%E7%AF%80%E7%B4%84/

        scheduler.step() # 学習率の更新

        # データローダーの合併

    allpreds = np.concatenate(allpreds)
    alltargets = np.concatenate(alltargets)

    # ロス関数はモニター用だけど、一応保管

    losses = np.mean(losses)

    # Score with rmse
    train_rme_loss = np.sqrt(mean_squared_error(alltargets,allpreds))

    return losses,train_rme_loss

In [ ]:
def validating(
    valid_dataloader,
    model
):
    
    model.eval()

    allpreds = []
    alltargets = []

    for a in valid_dataloader:

        losses = []

        with torch.no_grad():

            ids = a["ids"].to(device)
            mask = a["mask"].to(device)
            #tokentype = a["token_type_ids"].to(device)

            output = model(ids,mask)
            output = output["logits"].squeeze(-1)

            target = a["target"].to(device)

            loss = loss_fn(output,target)


            # 採点用に
            losses.append(loss.item())
            allpreds.append(output.detach().cpu().numpy())
            alltargets.append(target.detach().squeeze(-1).cpu().numpy())
            
            del loss


    # dataloader分を結合

    allpreds = np.concatenate(allpreds)
    alltargets = np.concatenate(alltargets)

    # lossは使わないけど一応回収

    losses = np.mean(losses)

    # rmseでスコア出し
    valid_rme_loss = np.sqrt(mean_squared_error(alltargets,allpreds))

    return allpreds,losses,valid_rme_loss

In [ ]:
bestscores = []

for fold in range(0,5):
    

    # initializing the data

    p_train = train[train["kfold"]!=fold].reset_index(drop=True)
    p_valid = train[train["kfold"]==fold].reset_index(drop=True)


    train_dataset = RobertaDataset(p_train["excerpt"],p_train["target"])
    valid_dataset = RobertaDataset(p_valid["excerpt"],p_valid["target"])

    train_dataloader = DataLoader(train_dataset,batch_size=train_batch,shuffle = True,num_workers=4,pin_memory=True)
    valid_dataloader = DataLoader(valid_dataset,batch_size=valid_batch,shuffle = False,num_workers=4,pin_memory=True)

    model = RobertaLastHiddenStateRegressor(MODEL)
    model.to(device)
    LR=2e-5
    optimizer = AdamW(model.parameters(), LR,betas=(0.9, 0.999), weight_decay=1e-2) # AdamW optimizer

    train_steps = int(len(p_train)/train_batch*epochs)

    num_steps = int(train_steps*0.1)

    scheduler = transformers.get_cosine_schedule_with_warmup(optimizer, num_steps, train_steps)


    trainlosses = []
    vallosses = []
    bestscore = None

    trainscores = []
    validscores = []

    for epoch in tqdm(range(epochs)):

        print("----------fold:" + str(fold) + "----" + "epoch:" + str(epoch) + "start-------------")

        trainloss,trainscore = training(train_dataloader,model,optimizer,scheduler)

        trainlosses.append(trainloss)
        trainscores.append(trainscore)

        print("trainscore is " + str(trainscore))

        preds,validloss,valscore=validating(valid_dataloader,model)

        vallosses.append(validloss)
        validscores.append(valscore)


        print("valscore is " + str(valscore))

        if bestscore is None:
            bestscore = valscore

            print("Save first model")

            state = {
                            'state_dict': model.state_dict(),
                            'optimizer_dict': optimizer.state_dict(),
                            "bestscore":bestscore
                        }


            torch.save(state, F"/content/drive/MyDrive/03-roberta-model" + str(fold) + ".pth")

        elif bestscore > valscore:

            bestscore = valscore

            print("found better point")

            state = {
                            'state_dict': model.state_dict(),
                            'optimizer_dict': optimizer.state_dict(),
                            "bestscore":bestscore
                        }


            torch.save(state, F"/content/drive/MyDrive/03-roberta-model"+ str(fold) + ".pth")

        else:
            pass

    bestscores.append(bestscore)

Some weights of the model checkpoint at /content/drive/MyDrive/02-roberta_pretrain_base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /content/drive/MyDrive/02-roberta_pretrain_base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRA

----------fold:0----epoch:0start-------------


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-s

trainscore is 1.2138649


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-s

...

valscore is 0.49524057
----------fold:4----epoch:99start-------------


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-s

trainscore is 0.05521995


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-s

valscore is 0.49526423


In [ ]:
bestscores

[0.5255655, 0.5165656, 0.50754845, 0.51618767, 0.46994326]

In [ ]:
np.mean(bestscores)
print("my cv is " + str(np.mean(bestscores)))

my cv is 0.50716215
